In [1]:
from pynq import Overlay, allocate, MMIO

In [2]:
import numpy as np
import time

In [3]:
# Load full bistream (static shell + RM)
overlay = Overlay('./design_1_wrapper.bit')

In [4]:
# BUG: PYNQ is not able to map the newly added RM into its IP dictionary
# WORKAROUND - AFTER PYNQ PATCHING: Direct MMIO is required to access the IP's S_AXI_CTRL (see HLS-generated drivers for register documentation)
rm = overlay.ip_dict['reconfigurable_module/s_axi_control']
rm_mmio = MMIO(rm['phys_addr'], rm['addr_range'])

In [5]:
# Reconfigurable modules for partition called 'reconfigurable_module'
#   operation:   the software implementation of the operation inside the RM
#   bit_path:    the path to the partial bitstream (and HWH) implementing the RM
pdr_mods = [
    {
        'operation': lambda a,b: a - b,
        'bit_path':'./design_1_i_reconfigurable_module_pr_subber_inst_0_partial.bit'
    }, 
    {
        'operation': lambda a,b: a + b,
        'bit_path': './design_1_i_reconfigurable_module_rp_adder_inst_0_partial.bit'
    }, 
    {
        'operation': lambda a,b: a * b,
        'bit_path': './design_1_i_reconfigurable_module_rm_multer_inst_0_partial.bit'
    }
]

In [7]:
# MAIN LOOP

N_ITERATIONS = 1000 # Set number of iterations
for iteration in range(N_ITERATIONS):
    for i in range(len(pdr_mods)):
        # Select a variant of the RM
        current_pdr_mod = pdr_mods[i % len(pdr_mods)]
        
        
        #1. DECOUPLE
        overlay.dfx_decoupler_0.mmio.write(0, 0b1)
        
        #2. WAIT FOR DECOUPLE (polling)
        #   FIXME: Use interrupts
        while overlay.dfx_decoupler_0.mmio.read(0) != 0b1:
            time.sleep(1e-6)
            
        #3. RECONFIGURE
        overlay.pr_download('reconfigurable_module', current_pdr_mod['bit_path'])
        
        #4. ASSERT RESET
        #   FIXME: Unimplemented
        
        #5. RECOUPLE
        overlay.dfx_decoupler_0.mmio.write(0, 0b0)
        
        #6. WAIT FOR RECOUPLE (polling)
        #   FIXME: Use interrupts
        while overlay.dfx_decoupler_0.mmio.read(0) != 0b0:
            time.sleep(1e-6)

        #7. DISASSERT RESET
        #   FIXME: Unimplemented
        
        # Setup buffers for DMA transfer
        # ⚠ Single sample mode
        the_as = allocate(shape=(1,),dtype=np.int32)
        the_bs = allocate(shape=(1,),dtype=np.int32)
        the_cs = allocate(shape=(1,),dtype=np.int32)
        the_as[0] = i
        the_bs[0] = i * 10
        the_cs[0] = -42
        the_as.flush()
        the_bs.flush()
        the_cs.flush()
        
        # Start DMAs
        overlay.axi_dma_c.recvchannel.transfer(the_cs)
        overlay.axi_dma_a.sendchannel.transfer(the_as)
        overlay.axi_dma_b.sendchannel.transfer(the_bs)

        # Start RM
        rm_mmio.write(0, 0b1)
        
        # WAIT FOR RM TO COMPLETE (polling)
        # FIXME: Use interrupts
        retry = 3
        while rm_mmio.read(0) & 0b10 != 0b10:
            retry -= 1
            if retry <=0:
                print('@', i, ' NOT RESPONDING')
                break
            rm_mmio.write(0, 0b1)
            time.sleep(1e-6)   
    
        # Read out the results and compare against software execution
        the_cs.invalidate()
        if the_cs[0] != current_pdr_mod['operation'](the_as[0], the_bs[0]):
            print(current_pdr_mod['bit_path'], '\'s output is wrong!')
            print('IN:', the_as, the_bs)
            print('OUT:', the_cs)
        else:
            print('OK!')


OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!
OK!


KeyboardInterrupt: 